In [31]:
import talib
import sqlite3
import pandas as pd
from datetime import datetime
from importlib import reload
import time
import numpy as np
import matplotlib.pyplot as plt
import connectorx as cx
from concurrent.futures import ThreadPoolExecutor, as_completed


In [11]:
import functions; reload(functions)
from functions import *

In [12]:
import sys
print(sys.executable)

/opt/homebrew/Caskroom/miniconda/base/envs/stonks/bin/python


In [13]:
conn = sqlite3.connect('main.db')
c = conn.cursor()

In [ ]:
tds = get_trading_days()

In [14]:
def datelist_to_df_parallel(datelist, max_workers=11, json=False):
    before = datetime.now()
    
    alljson = []
    
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [executor.submit(daily_agg, date) for date in datelist]
        for future in tqdm(as_completed(futures), total=len(datelist)):
            try:
                data = future.result()
                if data is not None:
                    alljson.extend(data)
            except Exception as e:
                print(f"Error fetching data: {e}")
    
    after = datetime.now()
    file_request_print(len(datelist), after - before)
    
    if len(alljson) == 0:
        return None
    elif json:
        return alljson
    else:
        return pd.DataFrame(alljson)

In [33]:
df1 = datelist_to_df(tds[:100])

100%|██████████| 100/100 [01:38<00:00,  1.01it/s]


100 file requests done in 1.0 minutes and 38.98 seconds


In [34]:
df2 = datelist_to_df_parallel(tds[:100])

100%|██████████| 100/100 [00:11<00:00,  8.73it/s]


100 file requests done in 11.466 seconds


In [35]:
df3 = datelist_to_df_parallel(tds[:100], max_workers=30)

  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [00:07<00:00, 14.03it/s]


100 file requests done in 7.1463 seconds


In [19]:
df.head()

,T,v,vw,o,c,h,l,t,n
0,IYK,99318.0,42.0951,42.0333,42.04,42.1667,42.01,1564516800000,242.0
1,RST,179599.0,23.2769,22.6100,23.27,23.5750,22.40,1564516800000,2192.0
2,MYJ,27073.0,15.0528,15.0600,15.07,15.0800,15.01,1564516800000,145.0
3,OXFD,1296839.0,12.7405,12.6600,12.66,12.9800,12.56,1564516800000,2693.0
4,PLAN,909264.0,57.6294,57.1600,57.69,58.4700,56.70,1564516800000,8709.0


In [26]:
df.to_csv('alldata.csv', index=False)

In [25]:
df = datelist_to_df(tds)

100%|██████████| 1257/1257 [26:39<00:00,  1.27s/it] 


1257 file requests done in 26.0 minutes and 39.68 seconds


In [78]:
maxtime = 'SELECT MAX(time) FROM stocks;'
c.execute(maxtime).fetchone()[0]

1720468800000

In [40]:
query = '''
SELECT * FROM stocks
WHERE ticker in (
	SELECT ticker FROM stocks
	WHERE time = (
		SELECT MAX(time) FROM stocks
	)
	ORDER BY volume * vw DESC
	LIMIT 5000
)
'''

In [69]:
bla = '''
SELECT ticker, close, time FROM stocks
WHERE ticker in (
SELECT ticker FROM stocks
	WHERE time = (
		SELECT MAX(time) FROM stocks
	)
    ORDER BY volume * vw DESC
    LIMIT 5000
  )
'''

In [41]:
data = get_data(query)

In [70]:
cxtest = cx.read_sql('sqlite://main.db', bla)

In [76]:
cx2 = cx.read_sql('sqlite://main.db', get_top_stocks())

In [35]:
test = pd.read_sql_query(query, conn)

In [38]:
test2 = pd.read_sql_query(query, conn)

In [16]:
timestamp_to_date(get_data(maxtime)[0][0])

'2024-07-03'

In [28]:
df = pd.read_sql_query(get_top_stocks(10000), conn)

In [17]:
df = get_data(get_top_stocks(10))

In [19]:
df = pd.DataFrame(df)

In [ ]:
onlylast = '''
    SELECT ticker FROM stocks
	WHERE time = (
		SELECT MAX(time) FROM stocks
	)
	ORDER BY volume * vw DESC
    LIMIT 1000;
'''

query = f'''
    SELECT * 
    FROM stocks 
    WHERE time > {timestamp_from_datestring('2024-01-01')}
    AND n > 1000
    '''
	
### test table
beginning = date_to_timestamp('2023-01-01')
end = date_to_timestamp('2024-06-01')
query = f'''
    SELECT * 
    FROM stocks 
    WHERE time > {beginning} AND time < {end}
'''
test = pd.read_sql_query(query, conn)
test.to_sql('test', conn, if_exists='replace', index=False)

In [6]:
def get_data(query):
    c.execute(query)
    data = c.fetchall()
    return data

In [56]:
timestamps = pd.read_sql_query(f'SELECT DISTINCT time FROM {tablename}', conn)
dates = [date_from_timestamp(ts) for ts in timestamps['time']]
trading_days = get_trading_days(from_date=min(dates))[1:]
missingdates = missingdates(dates, trading_days)